## CellCnn [1] data generation

source: https://github.com/eiriniar/CellCnn

""" Copyright 2016-2017 ETH Zurich, Eirini Arvaniti and Manfred Claassen.

This module contains data preprocessing/distribution functions.

"""
The code is slightly changed depending on original implementation to make it compatible with decentralized settings



In this example, we preprocess and distribute a mass Acute Myeloid Leukaemia (AML) dataset[2] for 3-class classification problem for healthy, cytogenetically normal (CN), and core-binding factor translocation (CBF). For each cell, the dataset includes mass cytometry measurements of 16 markers for each cell. 


The dataset comprises mass cytometry measurements of 16 markers, as in original cellCnn[1] analysis, we use the AML samples on the AML samples with at least 10% CD34+ blast cells with availability of additional cytogenetic information. 

To run this example, 

    - download the [AML cell dataset] at https://imsb.ethz.ch/research/claassen/Software/cellcnn.html, under ALL dataset zip folder
    - uncompress and place it in the data/cellCNN/ folder

Data distribution: We fix the test set for all experimental settings, the training dataset is then generated by distribution different donors for each institution depending on number of institutions.

[1] E. Arvaniti and M. Claassen. Sensitive detection of rare disease-associated cell subsets via representation learning.Nat Commun, 8:1–10, 2017
[2] . Levine, E. Simonds, S. Bendall, K. Davis, E.-A. Amir, M. Tadmor, O. Litvin, H. Fienberg, A. Jager, E. Zunder, R. Finck, A. Gedman,I. Radtke, J. Downing, D. Pe’er, and G. Nolan. Data-driven phenotypic dissection of aml reveals progenitor-like cells that correlatewith prognosis.Cell, 162, 2015.

In [ ]:
import os, sys, errno, glob
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import cellCNN_utils  
import sklearn.utils as sku
from cellCNN_utils import loadFCS, ftrans, mkdir_p, get_items, generate_data, generate_normalized_data
from pathlib import Path
%pylab inline

rand_seed = 12345
np.random.seed(rand_seed)
stim = 'AML'

d = Path().resolve()
sys.path.append(d)

# WDIR = os.path.join(cellCnn.__path__[0], 'examples')
OUTDIR = os.path.join(d, 'output_%s' % stim)
mkdir_p(OUTDIR)

LOOKUP_PATH = os.path.join(d, 'AML.pkl')

with open(LOOKUP_PATH, 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin'
    lookup = u.load()



In [ ]:
#patients SJ10, SJ12, SJ13 were characterized as CN
#patients SJ1, SJ2, SJ3, SJ4, SJ5 presented CBF
labels = ['CD19', 'CD11b', 'CD34', 'CD45', 'CD123', 'CD33', 'CD47', 'CD7', 'CD15', 'CD44', 'CD38', 'CD3', 'CD117', 'HLA-DR', 'CD64', 'CD41']
trainCN, trainCBF, trainHealthy = [], [],[]
testCN, testCBF, testHealthy = [], [],[]
for key, val in lookup.items():
    if "SJ" not in key: 
        if key == "healthy_BM":
            trainHealthy.append(val[0][1])
            trainHealthy.append(val[1][1])
            trainHealthy.append(val[2][1])
            testHealthy.append(val[3][1])
            testHealthy.append(val[4][1])

    if key == 'SJ10' or key == 'SJ12':
        trainCN.append(val)
    if key == 'SJ13':
        testCN.append(val)
    if key == 'SJ1' or key == 'SJ2': 
        trainCBF.append(val)
    if key == 'SJ3' or key == 'SJ4' or key == 'SJ5':
        testCBF.append(val)

test_samples = testCN+ testCBF+ testHealthy
test_phenotypes = [1,2,2,2,0,0 ] 

train_phenotypes = [0, 1 ,2]  #healthy, cn, cbf

x_trainHealthy = sku.shuffle(np.vstack(trainHealthy))
x_trainCN = sku.shuffle(np.vstack(trainCN))
x_trainCBF = sku.shuffle(np.vstack(trainCBF))

train_samples = [x_trainHealthy,x_trainCN, x_trainCBF]


### Generate original data (with transform)
In the following, 
- We generate training data with $ncell=200$ cells per sample and $nsubset=500$ samples per class
- We generate the test data for $ncell=200$ cells per sample from test indices, called X_test
- We generate another test set 'per-individual' in test indices using maximum number of cells to use for phenotype prediction, called X_test_all

Processed data is placed under originalAML/ folder

The script prints the max number of cells for the current example (i.e., 12440 for this dataset) which then will be used as a parameter in the golang protocol.

In [ ]:
from sklearn.utils import shuffle

scaler,x_tr,y_tr,x_test,y_test = generate_data(train_samples, train_phenotypes, 'originalAML/', valid_samples=test_samples, valid_phenotypes=test_phenotypes, ncell=200, nsubset=500, verbose=0, saveFile=True)

#generate also the test set on full ncell per sample:
def generate_for_pheno_prediction(new_samples,phenotypes,scaler):
        ncell_per_sample = np.min([x.shape[0] for x in new_samples])
        print(f"Predictions based on multi-cell inputs containing {ncell_per_sample} cells.")
        nmark = len(new_samples[0][1])
        # z-transform the new samples if we did that for the training samples
        if scaler is not None:
            new_samples = [scaler.transform(x) for x in new_samples]
        new_samples = [shuffle(x)[:ncell_per_sample].reshape(1, ncell_per_sample, nmark)
                           for x in new_samples]
        data_test = np.vstack(new_samples)
        mkdir_p('originalAML/X_test_all/')
        for i in range(len(data_test)):
            np.savetxt('originalAML/' + 'X_test_all/' + str(i) +'.txt', (transpose(data_test[i])))
        np.savetxt('originalAML/' + 'y_test_all.txt', phenotypes)
        return data_test,phenotypes

data_test,phenotypes=generate_for_pheno_prediction(test_samples,test_phenotypes,scaler)
print(shape(data_test))
print(shape(x_test))
print(shape(y_test))

### Generate  data split between $nhosts$ parties
In the following, 
- We generate training data with $ncell=200$ cells per sample and $nsubset=700$ samples per class, per party
- Example below distributes the train indices per donor for 2 parties

Processed data is placed under splitAML/host_i for party-i


In [ ]:
# set random seed for reproducible results
np.random.seed(1234)
# Here we randomly split the samples in training/test sets.
nhosts= 3


#distribute train indices balanced among n hosts:
numH = [0,1,2] #training set healthy sample indices
numCN = [0,1] #training set CN indices
numCBF = [0,1] #training set CBF indices
group1_list = np.flip(np.array_split(numpy.array(numH), nhosts))
group2_list = numpy.array_split(numpy.array(numCN), nhosts)
group3_list = numpy.array_split(numpy.array(numCBF), nhosts)

split_idx_1 = []
split_idx_2 = []
split_idx_3 = []
for i in range(nhosts):
    split_idx_1.append(group1_list[i].tolist())
    split_idx_2.append(group2_list[i].tolist())
    split_idx_3.append(group3_list[i].tolist())


print("Global train splitted among hosts - indices:")

#make sure each client gets at least one patient, comment in the following part to shuffle otherwise
# random.shuffle(split_idx_1)
# random.shuffle(split_idx_2)
# random.shuffle(split_idx_3)

#or manually distribute indices
split_idx_1 = [[0], [1], [2]]
split_idx_2 = [[0], [1], []]
split_idx_3 = [[1], [], [0]]
print(split_idx_1)
print(split_idx_2)
print(split_idx_3)
for i in range(nhosts):
    print("\nHost no.", i, ":")
    folder_path = 'splitAML/host' + str(i) + '/'
    trainHealthyidx = split_idx_1[i]
    trainCNTempidx = split_idx_2[i]
    trainCBFTempidx = split_idx_3[i]
    trainHealthyTemp,trainCNTemp,trainCBFTemp = [],[],[]
    for idx in trainHealthyidx:
        trainHealthyTemp.append(trainHealthy[idx])
    for idx in trainCNTempidx:
        trainCNTemp.append(trainCN[idx])
    for idx in trainCBFTempidx:
        trainCBFTemp.append(trainCBF[idx])
    train_phenotypes = []
    train_samples =[]
     # load the training samples
    if len(trainHealthyTemp) != 0:
        x_trainHealthy = sku.shuffle(np.vstack(trainHealthyTemp))
        train_phenotypes.append(0)
    if len(trainCNTemp) != 0:
        x_trainCN = sku.shuffle(np.vstack(trainCNTemp))
        train_phenotypes.append(1)
    if len(trainCBFTemp) != 0:
        x_trainCBF = sku.shuffle(np.vstack(trainCBFTemp))
        train_phenotypes.append(2)
    train_samples = [x_trainHealthy] + [x_trainCN]+ [x_trainCBF]
    generate_data(train_samples, train_phenotypes, folder_path, ncell=200, nsubset=160, verbose=0,generate_valid_set=False,per_sample=False)

In [ ]:
#The reproduction of original CellCnn model training without validation test and further analysis part
#This part is used for the comparison of accuracy/precision/recall/f-score of CellCnn with secure distributed version
#test on 200-cell multi-instances and full test set phenotype prediction
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tensorflow import keras
from keras.utils import to_categorical
from tensorflow.keras import layers, initializers, regularizers, optimizers, callbacks
from keras import backend as K

#repeat cellcnn original training on full train data

k = 200
ncell = 200
nfilter = 8
y_tr_n = to_categorical(y_tr)

def pool_top_k(x, k):
    return tf.reduce_mean(tf.sort(x, axis=1, direction='DESCENDING')[:, :k, :], axis=1)
def create_model (k,ncell,nfilter):
        
        data_input = keras.Input(shape=(ncell, 16))
        coeff_l1=0
        coeff_l2=1e-4
        n_classes=3
        # the filters
        conv = layers.Conv1D(filters=nfilter,
                             kernel_size=1,
                             activation='relu',
                             kernel_initializer=initializers.RandomUniform(),
                             kernel_regularizer=regularizers.l1_l2(l1=coeff_l1, l2=coeff_l2),
                             name='conv1')(data_input)

        # the cell grouping part (top-k pooling)
        pooled = layers.Lambda(pool_top_k, output_shape=(nfilter,), arguments={'k': k})(conv)
        output = layers.Dense(units=n_classes,
                                  activation='softmax',
                                  kernel_initializer=initializers.RandomUniform(),
                                  kernel_regularizer=regularizers.l1_l2(l1=coeff_l1, l2=coeff_l2),
                                  name='output')(pooled)
        model = keras.Model(inputs=data_input, outputs=output)

        model.compile(optimizer=optimizers.Adam(learning_rate=0.01),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])
        return model
model = create_model(k,ncell,nfilter)
#generate data

x_tr_n = x_tr.transpose(0,2,1)
# Fit data to model
print(type(x_tr_n))
print(len(x_tr_n))
print(len(x_tr_n[0]))
print(len(x_tr_n[1][0]))


history = model.fit(x_tr_n, y_tr_n,
            batch_size=200,
            epochs=20,
            verbose=2,
            validation_split=0)

In [ ]:
#For 200-cell predictions on test set
def model_pred(prob):
    pred = []
    for p in prob:
        max_value = max(p)
        max_index =  np.where(p == max_value)
        pred.append(max_index)
    pred = np.array(pred)
    pred=pred.reshape(len(prob))
    return pred

x_test_n = x_test.transpose(0,2,1)
y_test_n = to_categorical(y_test)

loss, accuracy = model.evaluate(x_test_n, y_test_n, verbose=0)
#score = model.evaluate(x_test_n, y_test_n, verbose=0)
print("For 200-cell predictions on test set with size",x_test_n.shape)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

y_pred = model.predict(x_test_n)
y_pred = model_pred(y_pred)


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F-score:",f1_score(y_test, y_pred, average="macro"))
print("precision:",precision_score(y_test, y_pred, average="macro"))
print("recall:",recall_score(y_test, y_pred, average="macro")) 



In [ ]:
#For phenotype predictions on test set using all cells 

model2 = create_model(12440, 12440,8)
weights = model.get_weights()
model2.set_weights(weights)
data_test_n = data_test.transpose(0,2,1)
phenotypes_n = to_categorical(phenotypes)

y_pred = model2.predict(data_test)

y_pred = model_pred(y_pred)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
print("Accuracy:", accuracy_score(phenotypes, y_pred))
print("F-score:",f1_score(phenotypes, y_pred, average="macro"))
print("precision:",precision_score(phenotypes, y_pred, average="macro"))
print("recall:",recall_score(phenotypes, y_pred, average="macro"))